# Lab 06 - Generative adversarial networks

## Introduction

In this laboratory we will implement what is called in the literature a Cycle GAN (https://arxiv.org/pdf/1703.10593.pdf). These are powerful models that can learn to translate images without having paired samples to learn from.

A Generative Adversarial Network can generate novel data that is similar to that found in the training set (comes from the same distribution). As you've seen in the lecture, classic models can take random noise as input and learn to generate such data. There are also models where, in addition to the random noise, the model can take another input which it is meant to translate. For instance, this paper https://arxiv.org/pdf/1611.07004.pdf showcases translating images containing outlines alone to fully fledged objects.

![](./edge_to_obj_.png)

In such cases, there are pairs of desired inputs and outputs. However, it is rarely the case that we can create or benefit from such well structured datasets. The general case consists in samples gathered separately from the two classes that we wish to translate. For instance you can have several images of horses, and several images of zebras with no way of pairing them.

![](./h2z.png)

A CycleGAN can learn to translate from one class to another given such a scenario.

## CycleGAN

A CycleGAN primarily consists in have two GAN models: one that translates from the first class to the second, and one that translates from the second class to the first. Lets name those classes X and Y.

There will be a generator that takes as input an image from the X class and outputs an image from the Y class, a discrimiator that has to distinguish real Y images from fake ones, a generator that translates from Y to X and a final distriminator that classifies real and fake X images.

In order to prevent the generators to output a single image from the opposite class regardless of the input there is an additional factor that the model has to optimize; the most important aspect of the architecture. The translations have to be cycle consistent. This means that if we take an X image and translate it into a Y image, and then translate it again into an X image, we should end up with the exact same image we started with. This is the fundamental ideea of this model.

![](./cycle.png)

Additionally, we will introduce another loss term. Given a X image, we what that the generator that translates from Y to X to exactly reproduce it (identity loss), similarly, the other way around.

## Implemenatation

With the help of your superviser you can implement a CycleGAN starting from this code skeleton. The example featured will be translating apples to oranges, this is so that a model trained for a few iterations can provide visually interesting results. The same model cand then be applied to the zebras to horses dataset, provided you train the model for a larger number of iterations, all you have to do is replace the 'apple2orange' dataset name to 'horse2zebra'. The script will load that dataset and train the model on it without any other changes required.

Firstly, importing all neccessary tools and loading the dataset.

In [1]:
#from tensorflow_examples.models.pix2pix import pix2pix
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf
import tensorflow_datasets as tfds
import time
from keras import layers

In addition to loading the dataset, we will also jitter and normalize the input in order to enhance our models performance.

In [2]:
def jitter(im, label):
    im = tf.image.resize(im, [286, 286])
    im = tf.image.random_crop(im, [256, 256, 3])
    # im = tf.image.resize(im, [128, 128])
    im = tf.image.random_flip_left_right(im)

    return im, label

def normalize(im, label):
    # im = tf.image.resize(im, [128, 128])
    return tf.cast(im, tf.float32) / 255.

dataset, metadata = tfds.load(
    'cycle_gan/apple2orange',
    with_info = True, as_supervised = True
)
train_x = dataset['trainA'].map(jitter).map(normalize).repeat().shuffle(1000).batch(10)
train_y = dataset['trainB'].map(jitter).map(normalize).repeat().shuffle(1000).batch(10)
test_x = dataset['testA'].map(normalize).batch(10)
test_y = dataset['testB'].map(normalize).batch(10)


Now, we will define our generator architecture. For this class we will use a simple U-Net generator. The ideea is to concatenate the convolutional features in the following manner.

![](./unet.jpg)

For instance, if your first layer has 32 filters, your last layer (excluding the final output) that will also have 32 layers, will be contatenated with the first one obtaining 64 channels as input for what follows. Another example suppose you have an architecture with the following number of filters: input -> 32 -> 64 -> 128 -> 64 -> 32 -> 3(final output). Then, the final 32-filter convolution output will have its output concatened with the output of the first 32-filter convolution, and the last 64-filter convolution will have its output concatenated with that of the first 64-filter convolution output. The reason for this is that the generators have to reconstruct pretty much the same visual structure they received as input so instead of having the task of memorizing the structure, we will simply give the the structure for it as it advances in its reconstruction.

Our final layer will have sigmoid activation in order to reconstruct the input.

In [3]:
class Generator(tf.keras.Model):
    def __init__(self):
        super(Generator, self).__init__()
        self.downscale_layers = [layers.Conv2D(64, 3, padding='same', input_shape=(256, 256, 3), activation='relu')]
        # define a list of convolutional layers with
        # 64, 128, 256 and 512 filters respectively
        # relu activations, same padding and a kernel of size 3
        filters = [128, 256, 512]
        for f in filters:
            self.downscale_layers.append(layers.Conv2D(f, 3, padding='same', activation='relu'))

            # define a 2x2 max pooling layer
        self.pool_layer = layers.MaxPool2D((2, 2))

        # define a list of transposed convolutions with
        # 256, 128 and 64 filters respectively
        # relu activations, same padding, a kernnel of size 3 and a stride of size 2
        transposed_filters = [256, 128, 64]
        self.transposed_convs = []
        for t_f in transposed_filters:
            self.transposed_convs.append(layers.Conv2DTranspose(t_f, 3,strides=(2, 2), padding='same', activation='relu'))

        # define a final deconvolution with 3 filters, kernel size 3, stride 2,
        # sigmoid activation, and same padding
        self.final_layer = layers.Conv2DTranspose(3, 3, strides=(2, 2), padding='same', activation='sigmoid')


    def call(self, input):
        # sequentially apply the convolutions and pooling layer
        # retain all max pooling outputs in a list
        intermed_results = []
        for layer in self.downscale_layers:
            input = layer(input)
            input = self.pool_layer(input)
            intermed_results.append(input)
        # sequentially apply all deconvolutions (except for the output layer)
        # concatenate each output with its correspoding convolutional output
        for i, tr_layer in enumerate(self.transposed_convs):
            output = tr_layer(input)
            input = tf.concat([output, intermed_results[-(i+2)]], axis=-1)

            #apply the final decovolution and return the output
        output = self.final_layer(input)
        return output

For the discrminator, since its will not have a complicated architecture, it will suffice to define a function that returns a sequential layer.

In [4]:
def new_discriminator():
    # return a sequential model with
    # 4 convolutions with max pooling with the same parameters as the ones in the generator
    # a dense layer with 128 units and relu activation
    # a final prediction fully connected layer with 1 unit and sigmoid activation
    return tf.keras.Sequential([
        layers.Conv2D(64, 3, padding='same', input_shape=(256, 256, 3), activation='relu'),
        layers.MaxPool2D((2, 2)),
        layers.Conv2D(128, 3, padding='same', activation='relu'),
        layers.MaxPool2D((2, 2)),
        layers.Conv2D(256, 3, padding='same',  activation='relu'),
        layers.MaxPool2D((2, 2)),
        layers.Conv2D(512, 3, padding='same',  activation='relu'),
        layers.MaxPool2D((2, 2)),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(1, activation='sigmoid')
    ])

We will instantiate all models and define their individual optimizers

In [5]:
gen_x_y = Generator()
gen_y_x = Generator()
disc_x = new_discriminator()
disc_y = new_discriminator()

opt_gen_x_y = tf.keras.optimizers.Adam(2e-4, beta_1 = 0.5)
opt_gen_y_x = tf.keras.optimizers.Adam(2e-4, beta_1 = 0.5)
opt_disc_x = tf.keras.optimizers.Adam(2e-4, beta_1 = 0.5)
opt_disc_y = tf.keras.optimizers.Adam(2e-4, beta_1 = 0.5)

The complex part comes next, defining the fit_iteration function. If you think it thoroughly, take it step by step and work with your supervisor, it should come naturally. As a special mention, whenever using gradient taping you should perform **all** (loss) computations within the context. Outside the context should only be gradient applications.

In [6]:
binary_cross_entropy = tf.keras.losses.BinaryCrossentropy()

def discriminator_loss(real, generated):
  real_loss = binary_cross_entropy(tf.ones_like(real), real)

  generated_loss = binary_cross_entropy(tf.zeros_like(generated), generated)

  total_disc_loss = real_loss + generated_loss

  return total_disc_loss * 0.5

def generator_loss(generated):
  return binary_cross_entropy(tf.ones_like(generated), generated)

def calc_cycle_loss(real_image, cycled_image):
  loss1 = tf.reduce_mean(tf.reduce_sum(tf.abs(real_image - cycled_image), axis=(1,2,3)))

  return 10 * loss1

def identity_loss(real_image, same_image):
  loss = tf.reduce_mean(tf.reduce_sum(tf.abs(real_image - same_image), axis=(1,2,3)))
  return 10 * 0.5 * loss

@tf.function
def fit_iteration(real_x, real_y):
    with tf.GradientTape(persistent=True) as tape:
        # watch variables from all 4 models
        tape.watch(gen_x_y.trainable_variables)
        tape.watch(gen_y_x.trainable_variables)
        tape.watch(disc_x.trainable_variables)
        tape.watch(disc_y.trainable_variables)

        # compute all need model outputs

        # generators applied on the real images
        fake_y = gen_x_y(real_x)
        fake_x = gen_y_x(real_y)
        # generators applied on the fake images (cycling)
        cycle_x = gen_y_x(fake_y)
        cycle_y = gen_x_y(fake_x)
        # generators applied for reconstructions (i.e. gen_x_y(real_y)) for the identity loss
        same_y = gen_x_y(real_y)
        same_x = gen_y_x(real_x)
        # discriminators applied on fake and real images
        disc_real_x = disc_x(real_x)
        disc_real_y = disc_y(real_y)

        disc_fake_x = disc_x(fake_x)
        disc_fake_y = disc_y(fake_y)

        # compute discriminator losses. Use mean binary cross entropy
        disc_x_loss = discriminator_loss(disc_real_x, disc_fake_x)
        disc_y_loss = discriminator_loss(disc_real_y, disc_fake_y)

        # compute generator losses. Use mean binary_cross_entropy
        gen_x_y_loss = generator_loss(disc_fake_y)
        gen_y_x_loss = generator_loss(disc_fake_x)
        # remember that the generator should trick the distriminator into thinking that the fake samples are real
        # should be cycle consistent
        # and should reproduce the input image if it is from the class that it should generate
        total_cycle_loss = calc_cycle_loss(real_x, cycle_x) + calc_cycle_loss(real_y, cycle_y)
        total_loss_gen_x_y = gen_x_y_loss + total_cycle_loss + identity_loss(real_x, same_x)
        total_loss_gen_y_x = gen_y_x_loss + total_cycle_loss + identity_loss(real_y, same_y)

        # compute and apply all gradients
    gen_x_y_gradients = tape.gradient(total_loss_gen_x_y, gen_x_y.trainable_variables)
    gen_y_x_gradients = tape.gradient(total_loss_gen_y_x, gen_y_x.trainable_variables)

    disc_y_gradients = tape.gradient(disc_y_loss, disc_y.trainable_variables)
    disc_x_gradients = tape.gradient(disc_x_loss, disc_x.trainable_variables)

    opt_gen_x_y.apply_gradients(zip(gen_x_y_gradients, gen_x_y.trainable_variables))
    opt_gen_y_x.apply_gradients(zip(gen_y_x_gradients, gen_y_x.trainable_variables))

    opt_disc_x.apply_gradients(zip(disc_x_gradients, disc_x.trainable_variables))
    opt_disc_y.apply_gradients(zip(disc_y_gradients, disc_y.trainable_variables))

The rest of the implementation is standard, trains the model for a few iterations and plots a few transformations applied on test images.

In [7]:
checkpoint_path = "./checkpoints/train"

ckpt = tf.train.Checkpoint(
            gen_x_y=gen_x_y,
            gen_y_x=gen_y_x,
            disc_x=disc_x,
            disc_y=disc_y)
ckpt_manager = tf.train.CheckpointManager(
            ckpt,
            checkpoint_path,
            max_to_keep=1)

In [9]:
train_x = iter(train_x)
train_y = iter(train_y)
n_iterations = 500
for i in range(n_iterations):
    fit_iteration(
        next(train_x), next(train_y)
    )
    print(i / n_iterations, end='\r')

NotFoundError: Graph execution error:

Detected at node 'generator_1/conv2d_transpose_4/conv2d_transpose' defined at (most recent call last):
    File "c:\Users\Ahmad Wali\anaconda3\envs\Deep_Learning\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\Ahmad Wali\anaconda3\envs\Deep_Learning\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "c:\Users\Ahmad Wali\anaconda3\envs\Deep_Learning\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "c:\Users\Ahmad Wali\anaconda3\envs\Deep_Learning\lib\site-packages\traitlets\config\application.py", line 992, in launch_instance
      app.start()
    File "c:\Users\Ahmad Wali\anaconda3\envs\Deep_Learning\lib\site-packages\ipykernel\kernelapp.py", line 712, in start
      self.io_loop.start()
    File "c:\Users\Ahmad Wali\anaconda3\envs\Deep_Learning\lib\site-packages\tornado\platform\asyncio.py", line 215,

Additionally, you can implement Instance Normalization (https://arxiv.org/pdf/1607.08022.pdf) and regularization techniques in your model in order to further enhance performances.

Even without those, provided the implementation has been successful, the following block should yield semi-realistc images, even after training for as few as 500 iterations with a batch size of 1. Try it out!

In [ ]:
test_x = iter(test_x)
test_y = iter(test_y)
import matplotlib.pyplot as plt
for _ in range(6):
    real_x, real_y = next(test_x), next(test_y)
    plt.imshow(
        np.concatenate([
            real_x, gen_x_y(real_x)
        ], axis = 2)[0]
    )
    plt.show()